# Atari Breakout in RLlib

In [ ]:
#!pip install ray[rllib]
#!pip install ipywidgets gputil
#!pip install gym[atari] autorom[accept-rom-license] ray[rllib] torch torchvision atari-py ale-py ale_py
#!pip install "gym[accept-rom-license, atari]"

In [ ]:
import gym
import ray
from ray import tune
import ale_py



if ray.is_initialized():
    ray.shutdown()
ray.init(num_cpus=20, num_gpus=1)
    #env = gym.make("ALE/Breakout-v5")

checkpoint_directory = '/home/fitz/ray_results/PPO_2024-04-25_12-53-51/PPO_ALE_Breakout-v5_8a226_00000_0_2024-04-25_12-53-51/checkpoint_000013'
config = {
    "env": "ALE/Breakout-v5",
    "num_workers": 19,
    "num_envs_per_worker": 4,
    "num_gpus": 1,
    "framework": "torch",  
    "gamma": 0.99,
    "train_batch_size": 5000,
    "rollout_fragment_length": "auto",
    "sgd_minibatch_size": 500,
    "num_sgd_iter": 10,
    "clip_param": 0.1,
    "entropy_coeff": 0.01,
}


#config = {
#        "env": "ALE/Breakout-v5",
#        "preprocessor_pref": None,
#        "gamma": 0.99,
#        "num_gpus": 1,
#        "num_workers": 19,
#        "num_envs_per_worker": 4,
#        "create_env_on_driver": True,
#        "lambda": 0.95,
#        "kl_coeff": 0.5,
#        "clip_rewards": True,
#        "clip_param": 0.1,
#        "vf_clip_param": 10.0,
#        "entropy_coeff": 0.01,
#        "rollout_fragment_length": "auto",
#        "sgd_minibatch_size": 500,
#        "num_sgd_iter": 10,
#        "batch_mode": "truncate_episodes",
#        "framework": "torch",
#}

stop_criteria = {
    "training_iteration": 5000,
    "episode_reward_mean": 200
}

results = tune.run(
    "PPO",
    config=config,
    stop=stop_criteria,
    checkpoint_at_end=True,
    checkpoint_freq=50,
    keep_checkpoints_num=10,  # Keep 10 checkpoints
    checkpoint_score_attr="episode_reward_mean",  # Retain checkpoints with the highest episode rewards
    restore=checkpoint_directory
)
    


In [ ]:
best_trial = results.get_best_trial("episode_reward_mean", "max", "last")
best_checkpoint = results.get_best_checkpoint(best_trial, "episode_reward_mean", "max")
print("Best checkpoint:", best_checkpoint)


In [ ]:
ray.shutdown()